# Copyright Netherlands eScience Center <br>
** Function     : Predict the Spatial Sea Ice Concentration with BayesConvLSTM at weekly time scale** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2020.03.02 ** <br>
** Last Update  : 2020.03.06 ** <br>
** Library      : Pytorth, Numpy, NetCDF4, os, iris, cartopy, dlacs, matplotlib **<br>
Description     : This notebook serves to predict the Arctic sea ice using deep learning. The Bayesian Convolutional Long Short Time Memory neural network is used to deal with this spatial-temporal sequence problem. We use Pytorch as the deep learning framework. <br>
<br>
** Here we predict sea ice concentration with one extra relevant field from either ocean or atmosphere to test the predictor.** <br>

Return Values   : Time series and figures <br>

The regionalization adopted here follows that of the MASIE (Multisensor Analyzed Sea Ice Extent) product available from the National Snow and Ice Data Center:<br>
https://nsidc.org/data/masie/browse_regions<br>
It is given by paper J.Walsh et. al., 2019. Benchmark seasonal prediction skill estimates based on regional indices.<br>

In [1]:
%matplotlib inline

import sys
import warnings
import numbers

# for data loading
import os
from netCDF4 import Dataset
# for pre-processing and machine learning
import numpy as np
import sklearn
#import scipy
import torch
import torch.nn.functional

#sys.path.append(os.path.join('C:','Users','nosta','ML4Climate','Scripts','DLACs'))
#sys.path.append("C:\\Users\\nosta\\ML4Climate\\Scripts\\DLACs")
sys.path.append("../")
import dlacs
import dlacs.BayesConvLSTM
import dlacs.preprocess
import dlacs.function

# for visualization
import dlacs.visual
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import iris # also helps with regriding
import cartopy
import cartopy.crs as ccrs

# ignore all the DeprecationWarnings by pytorch
if not sys.warnoptions:
    warnings.simplefilter("ignore")

The testing device is Dell Inspirion 5680 with Intel Core i7-8700 x64 CPU and Nvidia GTX 1060 6GB GPU.<br>
Here is a benchmark about cpu v.s. gtx 1060 <br>
https://www.analyticsindiamag.com/deep-learning-tensorflow-benchmark-intel-i5-4210u-vs-geforce-nvidia-1060-6gb/

In [2]:
# constants
constant = {'g' : 9.80616,      # gravititional acceleration [m / s2]
            'R' : 6371009,      # radius of the earth [m]
            'cp': 1004.64,      # heat capacity of air [J/(Kg*K)]
            'Lv': 2500000,      # Latent heat of vaporization [J/Kg]
            'R_dry' : 286.9,    # gas constant of dry air [J/(kg*K)]
            'R_vap' : 461.5,    # gas constant for water vapour [J/(kg*K)]
            'rho' : 1026,       # sea water density [kg/m3]
            }

################################################################################# 
#########                           datapath                             ########
#################################################################################
# please specify data path
datapath_ERAI = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ERA-Interim'
#datapath_ERAI = 'H:\\Creator_Zone\\Core_Database_DeepLearn\\ERA-Interim'
datapath_ORAS4 = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ORAS4'
#datapath_ORAS4 = 'H:\\Creator_Zone\\Core_Database_DeepLearn\\ORAS4'
datapath_ORAS4_mask = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ORAS4'
#datapath_ORAS4_mask = 'H:\\Creator_Zone\\Core_Database_DeepLearn\\ORAS4'
#datapath_PIOMASS = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/PIOMASS'
#datapath_PIOMASS = 'H:\\Creator_Zone\\Core_Database_AMET_OMET_reanalysis\\PIOMASS'
#datapath_clim_index = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/Climate_index'
#datapath_clim_index = 'F:\\PhD_essential\\Core_Database_AMET_OMET_reanalysis\\Climate_index'
output_path = '/home/ESLT0068/NLeSC/Computation_Modeling/ML4Climate/PredictArctic/BayesMaps'
#output_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps'

In [3]:
if __name__=="__main__":
    print ('*********************** get the key to the datasets *************************')
    # weekly variables on ERAI grid
    dataset_ERAI_fields_sic = Dataset(os.path.join(datapath_ERAI,
                                      'sic_weekly_erai_1979_2017.nc'))
#     dataset_ERAI_fields_slp = Dataset(os.path.join(datapath_ERAI,
#                                       'slp_weekly_erai_1979_2017.nc'))
#     dataset_ERAI_fields_t2m = Dataset(os.path.join(datapath_ERAI,
#                                       't2m_weekly_erai_1979_2017.nc'))
#     dataset_ERAI_fields_z500 = Dataset(os.path.join(datapath_ERAI,
#                                        'z500_weekly_erai_1979_2017.nc'))
#     dataset_ERAI_fields_z850 = Dataset(os.path.join(datapath_ERAI,
#                                        'z850_weekly_erai_1979_2017.nc'))
#     dataset_ERAI_fields_uv10m = Dataset(os.path.join(datapath_ERAI,
#                                        'uv10m_weekly_erai_1979_2017.nc'))
#     dataset_ERAI_fields_rad = Dataset(os.path.join(datapath_ERAI,
#                                         'rad_flux_weekly_erai_1979_2017.nc'))
    #dataset_PIOMASS_siv = Dataset(os.path.join(datapath_PIOMASS,
    #                             'siv_monthly_PIOMASS_1979_2017.nc'))
    # OHC interpolated on ERA-Interim grid
    dataset_ORAS4_OHC = Dataset(os.path.join(datapath_ORAS4,
                                'ohc_monthly_oras2erai_1978_2017.nc'))
#     dataset_index = Dataset(os.path.join(datapath_clim_index,
#                             'index_climate_monthly_regress_1950_2017.nc'))
    #dataset_ERAI_fields_flux = Dataset(os.path.join(datapath_ERAI_fields,
    #                                  'surface_erai_monthly_regress_1979_2017_radiation.nc'))
    # mask
    dataset_ORAS4_mask = Dataset(os.path.join(datapath_ORAS4_mask, 'mesh_mask.nc'))
    print ('*********************** extract variables *************************')
    #################################################################################
    #########                        data gallery                           #########
    #################################################################################
    # we use time series from 1979 to 2016 (468 months in total)
    # training data: 1979 - 2013
    # validation: 2014 - 2016
    # variables list:
    # SIC (ERA-Interim) / SIV (PIOMASS) / SST (ERA-Interim) / ST (ERA-Interim) / OHC (ORAS4) / AO-NAO-AMO-NINO3.4 (NOAA)
    # integrals from spatial fields cover the area from 20N - 90N (4D fields [year, month, lat, lon])
    # *************************************************************************************** #
    # SIC (ERA-Interim) - benckmark
    SIC_ERAI = dataset_ERAI_fields_sic.variables['sic'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
    year_ERAI = dataset_ERAI_fields_sic.variables['year'][:-1]
    week_ERAI = dataset_ERAI_fields_sic.variables['week'][:]
    latitude_ERAI = dataset_ERAI_fields_sic.variables['latitude'][:]
    longitude_ERAI = dataset_ERAI_fields_sic.variables['longitude'][:]
    # T2M (ERA-Interim)
#     T2M_ERAI = dataset_ERAI_fields_t2m.variables['t2m'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
#     year_ERAI_t2m = dataset_ERAI_fields_t2m.variables['year'][:-1]
#     week_ERAI_t2m = dataset_ERAI_fields_t2m.variables['week'][:]
#     latitude_ERAI_t2m = dataset_ERAI_fields_t2m.variables['latitude'][:]
#     longitude_ERAI_t2m = dataset_ERAI_fields_t2m.variables['longitude'][:]
    # SLP (ERA-Interim)
#     SLP_ERAI = dataset_ERAI_fields_slp.variables['slp'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
#     year_ERAI_slp = dataset_ERAI_fields_slp.variables['year'][:-1]
#     week_ERAI_slp = dataset_ERAI_fields_slp.variables['week'][:]
#     latitude_ERAI_slp = dataset_ERAI_fields_slp.variables['latitude'][:]
#     longitude_ERAI_slp = dataset_ERAI_fields_slp.variables['longitude'][:]
    # Z500 (ERA-Interim)
#     Z500_ERAI = dataset_ERAI_fields_z500.variables['z'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
#     year_ERAI_z500 = dataset_ERAI_fields_z500.variables['year'][:-1]
#     week_ERAI_z500 = dataset_ERAI_fields_z500.variables['week'][:]
#     latitude_ERAI_z500 = dataset_ERAI_fields_z500.variables['latitude'][:]
#     longitude_ERAI_z500 = dataset_ERAI_fields_z500.variables['longitude'][:]
    # Z850 (ERA-Interim)
#     Z850_ERAI = dataset_ERAI_fields_z850.variables['z'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
#     year_ERAI_z850 = dataset_ERAI_fields_z850.variables['year'][:-1]
#     week_ERAI_z850 = dataset_ERAI_fields_z850.variables['week'][:]
#     latitude_ERAI_z850 = dataset_ERAI_fields_z850.variables['latitude'][:]
#     longitude_ERAI_z850 = dataset_ERAI_fields_z850.variables['longitude'][:]
    # UV10M (ERA-Interim)
#     U10M_ERAI = dataset_ERAI_fields_uv10m.variables['u10m'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
#     V10M_ERAI = dataset_ERAI_fields_uv10m.variables['v10m'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
#     year_ERAI_uv10m = dataset_ERAI_fields_uv10m.variables['year'][:-1]
#     week_ERAI_uv10m = dataset_ERAI_fields_uv10m.variables['week'][:]
#     latitude_ERAI_uv10m = dataset_ERAI_fields_uv10m.variables['latitude'][:]
#     longitude_ERAI_uv10m = dataset_ERAI_fields_uv10m.variables['longitude'][:]
    # SFlux (ERA-Interim)
#     SFlux_ERAI = dataset_ERAI_fields_rad.variables['SFlux'][:-1,:,:,:] # 4D fields [year, week, lat, lon]
#     year_ERAI_SFlux = dataset_ERAI_fields_rad.variables['year'][:-1]
#     week_ERAI_SFlux = dataset_ERAI_fields_rad.variables['week'][:]
#     latitude_ERAI_SFlux = dataset_ERAI_fields_rad.variables['latitude'][:]
#     longitude_ERAI_SFlux = dataset_ERAI_fields_rad.variables['longitude'][:]
    #SIV (PIOMASS)
    #SIV_PIOMASS = dataset_PIOMASS_siv.variables['SIV'][:-12]
    #year_SIV = dataset_PIOMASS_siv.variables['year'][:-1]
    # OHC (ORAS4)
    # from 1978 - 2017 (for interpolation) / from 90 N upto 40 N
    OHC_300_ORAS4 = dataset_ORAS4_OHC.variables['OHC'][:-1,:,:67,:]/1000 # unit Peta Joule
    latitude_ORAS4 = dataset_ORAS4_OHC.variables['latitude'][:]
    longitude_ORAS4 = dataset_ORAS4_OHC.variables['longitude'][:]
    mask_OHC = np.ma.getmask(OHC_300_ORAS4[0,0,:,:])
    # AO-NAO-AMO-NINO3.4 (NOAA)
#     AO = dataset_index.variables['AO'][348:-1] # from 1979 - 2017
#     NAO = dataset_index.variables['NAO'][348:-1]
#     NINO = dataset_index.variables['NINO'][348:-1]
#     AMO = dataset_index.variables['AMO'][348:-1]
#     PDO = dataset_index.variables['PDO'][348:-1]

*********************** get the key to the datasets *************************
*********************** extract variables *************************


In [4]:
    #################################################################################
    ###########                 global land-sea mask                      ###########
    #################################################################################
    sea_ice_mask_global = np.ones((len(latitude_ERAI),len(longitude_ERAI)),dtype=float)
    sea_ice_mask_global[SIC_ERAI[0,0,:,:]==-1] = 0
    #################################################################################
    ###########                regionalization sea mask                   ###########
    #################################################################################
    print ('*********************** create mask *************************')
    # W:-156 E:-124 N:80 S:67
    mask_Beaufort = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    # W:-180 E:-156 N:80 S:66
    mask_Chukchi = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    # W:146 E:180 N:80 S:67
    mask_EastSiberian = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    # W:100 E:146 N:80 S:67
    mask_Laptev = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    # W:60 E:100 N:80 S:67
    mask_Kara = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    # W:18 E:60 N:80 S:64
    mask_Barents = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    # W:-44 E:18 N:80 S:55
    mask_Greenland = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    # W:-180 E:180 N:90 S:80
    mask_CenArctic = np.zeros((len(latitude_ERAI),len(longitude_ERAI)),dtype=int)
    print ('*********************** calc mask *************************')
    mask_Beaufort[13:31,32:76] = 1

    mask_Chukchi[13:32,0:32] = 1
    mask_Chukchi[13:32,-1] = 1

    mask_EastSiberian[13:31,434:479] = 1

    mask_Laptev[13:31,374:434] = 1

    mask_Kara[13:31,320:374] = 1

    mask_Barents[13:36,264:320] = 1

    mask_Greenland[13:47,179:264] = 1
    mask_Greenland[26:47,240:264] = 0

    mask_CenArctic[:13,:] = 1
    print ('*********************** packing *************************')
    mask_dict = {'Beaufort': mask_Beaufort[:,:],
                 'Chukchi': mask_Chukchi[:,:],
                 'EastSiberian': mask_EastSiberian[:,:],
                 'Laptev': mask_Laptev[:,:],
                 'Kara': mask_Kara[:,:],
                 'Barents': mask_Barents[:,:],
                 'Greenland': mask_Greenland[:,:],
                 'CenArctic': mask_CenArctic[:,:]}
    seas_namelist = ['Beaufort','Chukchi','EastSiberian','Laptev',
                     'Kara', 'Barents', 'Greenland','CenArctic']
    #################################################################################
    ########                  temporal interpolation matrix                  ########
    #################################################################################
    # interpolate from monthly to weekly
    # original monthly data will be taken as the last week of the month
    OHC_300_ORAS4_weekly_series = np.zeros(SIC_ERAI.reshape(len(year_ERAI)*48,len(latitude_ERAI),len(longitude_ERAI)).shape,
                                           dtype=float)
    OHC_300_ORAS4_series= dlacs.preprocess.operator.unfold(OHC_300_ORAS4)
    # calculate the difference between two months
    OHC_300_ORAS4_deviation_series = (OHC_300_ORAS4_series[1:,:,:] - OHC_300_ORAS4_series[:-1,:,:]) / 4
    for i in np.arange(4):
        OHC_300_ORAS4_weekly_series[3-i::4,:,:] = OHC_300_ORAS4_series[12:,:,:] - i * OHC_300_ORAS4_deviation_series[11:,:,:]

    print ('******************  calculate extent from spatial fields  *******************')
    # size of the grid box
    dx = 2 * np.pi * constant['R'] * np.cos(2 * np.pi * latitude_ERAI /
                                            360) / len(longitude_ERAI)
    dy = np.pi * constant['R'] / 480
    # calculate the sea ice area
    SIC_ERAI_area = np.zeros(SIC_ERAI.shape, dtype=float)
#     SFlux_ERAI_area = np.zeros(SFlux_ERAI.shape, dtype=float)
    for i in np.arange(len(latitude_ERAI[:])):
        # change the unit to terawatt
        SIC_ERAI_area[:,:,i,:] = SIC_ERAI[:,:,i,:]* dx[i] * dy / 1E+6 # unit km2
#         SFlux_ERAI_area[:,:,i,:] = SFlux_ERAI[:,:,i,:]* dx[i] * dy / 1E+12 # unit TeraWatt
    SIC_ERAI_area[SIC_ERAI_area<0] = 0 # switch the mask from -1 to 0
    print ('================  reshape input data into time series  =================')
    SIC_ERAI_area_series = dlacs.preprocess.operator.unfold(SIC_ERAI_area)
#     T2M_ERAI_series = dlacs.preprocess.operator.unfold(T2M_ERAI)
#     SLP_ERAI_series = dlacs.preprocess.operator.unfold(SLP_ERAI)
#     Z500_ERAI_series = dlacs.preprocess.operator.unfold(Z500_ERAI)
#     Z850_ERAI_series = dlacs.preprocess.operator.unfold(Z850_ERAI)
#     U10M_ERAI_series = dlacs.preprocess.operator.unfold(U10M_ERAI)
#     V10M_ERAI_series = dlacs.preprocess.operator.unfold(V10M_ERAI)
#     SFlux_ERAI_area_series = dlacs.preprocess.operator.unfold(SFlux_ERAI_area)
    print ('******************  choose the fields from target region  *******************')
    # select land-sea mask
    sea_ice_mask_barents = sea_ice_mask_global[12:36,264:320]
    print ('******************  choose the fields from target region  *******************')
    # select the area between greenland and ice land for instance 60-70 N / 44-18 W
    sic_exp = SIC_ERAI_area_series[:,12:36,264:320]
#     t2m_exp = T2M_ERAI_series[:,12:36,264:320]
#     slp_exp = SLP_ERAI_series[:,12:36,264:320]
#     z500_exp = Z500_ERAI_series[:,12:36,264:320]
#     z850_exp = Z850_ERAI_series[:,12:36,264:320]
#     u10m_exp = U10M_ERAI_series[:,12:36,264:320]
#     v10m_exp = V10M_ERAI_series[:,12:36,264:320]
#     sflux_exp = SFlux_ERAI_area_series[:,12:36,264:320]
    ohc_exp = OHC_300_ORAS4_weekly_series[:,12:36,264:320]
    print(sic_exp.shape)
#     print(t2m_exp.shape)
#     print(slp_exp.shape)
#     print(z500_exp.shape)
#     print(u10m_exp.shape)
#     print(v10m_exp.shape)
#     print(sflux_exp.shape)
    print(ohc_exp.shape)
    print(latitude_ERAI[12:36])
    print(longitude_ERAI[264:320])
    print(latitude_ORAS4[12:36])
    print(longitude_ORAS4[264:320])
    #print(latitude_ERAI[26:40])
    #print(longitude_ERAI[180:216])
    #print(sic_exp[:])
    print ('*******************  pre-processing  *********************')
    print ('=========================   normalize data   ===========================')
    sic_exp_norm = dlacs.preprocess.operator.normalize(sic_exp)
#     t2m_exp_norm = deepclim.preprocess.operator.normalize(t2m_exp)
#     slp_exp_norm = deepclim.preprocess.operator.normalize(slp_exp)
#     z500_exp_norm = deepclim.preprocess.operator.normalize(z500_exp)
#     z850_exp_norm = deepclim.preprocess.operator.normalize(z850_exp)
#     u10m_exp_norm = deepclim.preprocess.operator.normalize(u10m_exp)
#     v10m_exp_norm = deepclim.preprocess.operator.normalize(v10m_exp)
#     sflux_exp_norm = deepclim.preprocess.operator.normalize(sflux_exp)
    ohc_exp_norm = dlacs.preprocess.operator.normalize(ohc_exp)
    print('================  save the normalizing factor  =================')
    sic_max = np.amax(sic_exp)
    sic_min = np.amin(sic_exp)
    print(sic_max,"km2")
    print(sic_min,"km2")
    print ('====================    A series of time (index)    ====================')
    _, yy, xx = sic_exp_norm.shape # get the lat lon dimension
    year = np.arange(1979,2017,1)
    year_cycle = np.repeat(year,48)
    month_cycle = np.repeat(np.arange(1,13,1),4)
    month_cycle = np.tile(month_cycle,len(year)+1) # one extra repeat for lead time dependent prediction
    month_cycle.astype(float)
    month_2D = np.repeat(month_cycle[:,np.newaxis],yy,1)
    month_exp = np.repeat(month_2D[:,:,np.newaxis],xx,2)
    print ('===================  artificial data for evaluation ====================')
    # calculate climatology of SIC
#     seansonal_cycle_SIC = np.zeros(48,dtype=float)
#     for i in np.arange(48):
#         seansonal_cycle_SIC[i] = np.mean(SIC_ERAI_sum_norm[i::48],axis=0)
    # weight for loss
#     weight_month = np.array([0,1,1,
#                              1,0,0,
#                              1,1,1,
#                              0,0,0])
    #weight_loss = np.repeat(weight_month,4)
    #weight_loss = np.tile(weight_loss,len(year))

*********************** create mask *************************
*********************** calc mask *************************
*********************** packing *************************
******************  calculate extent from spatial fields  *******************
================  reshape input data into time series  =================
******************  choose the fields from target region  *******************
******************  choose the fields from target region  *******************
(1824, 24, 56)
(1824, 24, 56)
[80.5  79.75 79.   78.25 77.5  76.75 76.   75.25 74.5  73.75 73.   72.25
 71.5  70.75 70.   69.25 68.5  67.75 67.   66.25 65.5  64.75 64.   63.25]
[18.   18.75 19.5  20.25 21.   21.75 22.5  23.25 24.   24.75 25.5  26.25
 27.   27.75 28.5  29.25 30.   30.75 31.5  32.25 33.   33.75 34.5  35.25
 36.   36.75 37.5  38.25 39.   39.75 40.5  41.25 42.   42.75 43.5  44.25
 45.   45.75 46.5  47.25 48.   48.75 49.5  50.25 51.   51.75 52.5  53.25
 54.   54.75 55.5  56.25 57.   57.75 58.5  5

# Procedure for LSTM <br>
** We use Pytorth to implement LSTM neural network with time series of climate data. ** <br>

In [5]:
    print ('*******************  parameter for check  *********************')
    choice_exp_norm = ohc_exp_norm
    print ('*******************  create basic dimensions for tensor and network  *********************')
    # specifications of neural network
    input_channels = 3
    hidden_channels = [3, 2, 1] # number of channels & hidden layers, the channels of last layer is the channels of output, too
    #hidden_channels = [3, 3, 3, 3, 2]
    #hidden_channels = [2]
    kernel_size = 3
    # here we input a sequence and predict the next step only
    #step = 1 # how many steps to predict ahead
    #effective_step = [0] # step to output
    batch_size = 1
    #num_layers = 1
    learning_rate = 0.01
    num_epochs = 150#0
    print ('*******************  cross validation and testing data  *********************')
    # take 10% data as cross-validation data
    cross_valid_year = 4
    # take 10% years as testing data
    test_year = 4
    # minibatch
    #iterations = 3 # training data divided into 3 sets
    print ('*******************  check the environment  *********************')
    print ("Pytorch version {}".format(torch.__version__))
    # check if CUDA is available
    use_cuda = torch.cuda.is_available()
    print("Is CUDA available? {}".format(use_cuda))
    # CUDA settings torch.__version__ must > 0.4
    # !!! This is important for the model!!! The first option is gpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   
    print ('*******************  run BayesConvLSTM  *********************')
    print ('The model is designed to make many to one prediction.')
    print ('A series of multi-chanel variables will be input to the model.')
    print ('The model learns by verifying the output at each timestep.')
    # check the sequence length
    sequence_len, height, width = sic_exp_norm.shape
    # initialize our model
    model = dlacs.BayesConvLSTM.BayesConvLSTM(input_channels, hidden_channels, kernel_size).to(device)
    # use Evidence Lower Bound (ELBO) to quantify the loss
    ELBO = dlacs.function.ELBO(height*width)
    # for classification, target must be integers (label)
    #ELBO = dlacs.function.ELBO(height*width,loss_function=torch.nn.KLDivLoss())
    #ELBO = dlacs.function.ELBO(height*width,loss_function=torch.CrossEntropyLoss())
    #ELBO = dlacs.function.ELBO(height*width,loss_function=torch.NLLLoss(reduction='mean'))
    # penalty for kl
    penalty_kl = sequence_len
    # stochastic gradient descent
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    # Adam optimizer
    optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(model)
    print(ELBO)
    print(optimiser)

*******************  parameter for check  *********************
*******************  create basic dimensions for tensor and network  *********************
*******************  cross validation and testing data  *********************
*******************  check the environment  *********************
Pytorch version 1.1.0
Is CUDA available? False
*******************  run BayesConvLSTM  *********************
The model is designed to make many to one prediction.
A series of multi-chanel variables will be input to the model.
The model learns by verifying the output at each timestep.
BayesConvLSTM(
  (cell0): BayesConvLSTMCell()
  (cell1): BayesConvLSTMCell()
  (cell2): BayesConvLSTMCell()
)
ELBO(
  (loss_function): MSELoss()
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)


In [6]:
    for name, param in model.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

cell0.Wxi_mu
tensor([[[[-1.0558e-01, -1.8014e-01,  3.3321e-02],
          [ 2.3716e-02, -1.2316e-01,  1.5326e-01],
          [-1.4931e-01, -1.2037e-01, -2.9395e-03]],

         [[-1.6941e-01,  1.2226e-01, -6.5187e-02],
          [-1.0834e-01, -1.4857e-01, -1.3921e-04],
          [-1.3112e-01,  1.2654e-01, -6.7747e-02]],

         [[ 1.7094e-01, -4.0709e-02,  1.6852e-01],
          [ 6.5585e-03, -1.6063e-01,  1.8219e-01],
          [ 1.2504e-01,  9.6949e-02, -7.3622e-02]]],


        [[[ 1.3706e-01, -1.2122e-01,  6.7758e-02],
          [ 8.9354e-02,  1.5358e-01,  1.2304e-01],
          [-6.3727e-02,  7.3968e-02, -1.8771e-01]],

         [[ 1.2527e-01, -1.1710e-02, -7.6872e-02],
          [ 3.5652e-03,  1.4838e-01, -5.1766e-02],
          [ 1.2488e-02,  1.5517e-02, -1.2831e-02]],

         [[ 1.7406e-01,  1.8051e-01,  4.7447e-02],
          [ 9.8560e-02, -1.8229e-03, -1.1217e-01],
          [ 5.5913e-03,  4.4141e-03, -1.0908e-01]]],


        [[[ 4.8881e-02, -5.0010e-02,  1.7787e-01],
  

In [ ]:
    print('##############################################################')
    print('#############  preview model parameters matrix  ###############')
    print('##############################################################')
    print('Number of parameter matrices: ', len(list(model.parameters())))
    for i in range(len(list(model.parameters()))):
        print(list(model.parameters())[i].size())

In [ ]:
    %%time
    print('##############################################################')
    print('##################  start training loop  #####################')
    print('##############################################################')
    # track training loss
    hist = np.zeros(num_epochs)
    # loop of epoch
    for t in range(num_epochs):
        # Clear stored gradient
        model.zero_grad()
        # loop of timestep
        for timestep in range(sequence_len - cross_valid_year*12*4 - test_year*12*4):
            # hidden state re-initialized inside the model when timestep=0
            #################################################################################
            ########          create input tensor with multi-input dimension         ########
            #################################################################################
            # create variables
            x_input = np.stack((sic_exp_norm[timestep,:,:],
                                choice_exp_norm[timestep,:,:],
                                month_exp[timestep,:,:])) #vstack,hstack,dstack
            x_var = torch.autograd.Variable(torch.Tensor(x_input).view(-1,input_channels,height,width)).to(device)
            #################################################################################
            ########       create training tensor with multi-input dimension         ########
            #################################################################################
            y_train_stack = sic_exp_norm[timestep+1,:,:] #vstack,hstack,dstack
            y_var = torch.autograd.Variable(torch.Tensor(y_train_stack).view(-1,hidden_channels[-1],height,width)).to(device)
            #################################################################################   
            # Forward pass
            y_pred, kl_loss, _ = model(x_var, timestep)
            # choose training data
            y_target = y_var
            # torch.nn.functional.mse_loss(y_pred, y_train) can work with (scalar,vector) & (vector,vector)
            # Please Make Sure y_pred & y_train have the same dimension
            # accumulate loss
            #print (timestep)
            if timestep == 0:
                loss = ELBO(y_pred, y_target, kl_loss,
                            1 / (len(hidden_channels) * 8 * penalty_kl)) # weight of KL due to 8 gates in each layer
            else:
                loss += ELBO(y_pred, y_target, kl_loss,
                             1 / (len(hidden_channels) * 8 * penalty_kl))            
        #print(y_pred.shape)
        #print(y_train.shape)
        # print loss at certain iteration
        if t % 5 == 0:
            print("Epoch ", t, "MSE: ", loss.item())
            # gradient check
            # Gradcheck requires double precision numbers to run
            #res = torch.autograd.gradcheck(loss_fn, (y_pred.double(), y_train.double()), eps=1e-6, raise_exception=True)
            #print(res)
        hist[t] = loss.item()

        # Zero out gradient, else they will accumulate between epochs
        optimiser.zero_grad()
    
        # Backward pass
        loss.backward()

        # Update parametersdd
        optimiser.step()
        
    # save the model
    # (recommended) save the model parameters only
    torch.save(model.state_dict(), os.path.join(output_path,'bayesconvlstm.pkl'))
    # save the entire model
    # torch.save(model, os.path.join(output_path,'bayesconvlstm.pkl'))

##############################################################
##################  start training loop  #####################
##############################################################
Epoch  0 MSE:  89783.515625
Epoch  5 MSE:  59760.1484375
Epoch  10 MSE:  39905.62890625
Epoch  15 MSE:  33778.2265625
Epoch  20 MSE:  29385.525390625
Epoch  25 MSE:  26195.1796875
Epoch  30 MSE:  19809.05859375
Epoch  35 MSE:  18534.54296875
Epoch  40 MSE:  16601.708984375
Epoch  45 MSE:  15840.9091796875
Epoch  50 MSE:  15043.3095703125
Epoch  55 MSE:  14217.677734375
Epoch  60 MSE:  13625.8056640625
Epoch  65 MSE:  13082.3955078125
Epoch  70 MSE:  12588.7265625


In [ ]:
    print ("*******************  Loss with time  **********************")
    fig00 = plt.figure()
    try:
        plt.plot(hist, label="Training loss")
        plt.legend()
        plt.show()
        fig00.savefig(os.path.join(output_path,'SIC_ERAI_LSTM_pred_error.png'),dpi=200)
    except:
        print('Model is reloaded instead of trained!')

In [ ]:
    print ('*******************  evaluation matrix  *********************')
    # The prediction will be evaluated through RMSE against climatology
    
    # error score for temporal-spatial fields, without keeping spatial pattern
    def RMSE(x,y):
        """
        Calculate the RMSE. x is input series and y is reference series.
        It calculates RMSE over the domain, not over time. The spatial structure
        will not be kept.
        Parameter
        ----------------------
        x: input time series with the shape [time, lat, lon]
        """
        x_series = x.reshape(x.shape[0],-1)
        y_series = y.reshape(y.shape[0],-1)
        rmse = np.sqrt(np.mean((x_series - y_series)**2,1))
        rmse_std = np.sqrt(np.std((x_series - y_series)**2,1))
    
        return rmse, rmse_std
    
    # error score for temporal-spatial fields, keeping spatial pattern
    def MAE(x,y):
        """
        Calculate the MAE. x is input series and y is reference series.
        It calculate MAE over time and keeps the spatial structure.
        """
        mae = np.mean(np.abs(x-y),0)
        
        return mae

In [ ]:
    %%time
    #################################################################################
    ########  operational lead time dependent prediction with testing data   ########
    #################################################################################
    print('##############################################################')
    print('###################  start prediction loop ###################')
    print('##############################################################')
    # the model learn from time series and try to predict the next time step based on the previous time series
    print ('*******************************  one step ahead forecast  *********************************')
    print ('************  the last {} years of total time series are treated as test data  ************'.format(test_year))
    # time series before test data
    pred_base_sic = sic_exp_norm[:-test_year*12*4,:,:]
    # predict x steps ahead
    step_lead = 16 # unit week
    # create a matrix for the prediction
    lead_pred_sic = np.zeros((test_year*12*4,step_lead,height,width),dtype=float) # dim [predict time, lead time, lat, lon]
    # start the prediction loop
    for step in range(test_year*12*4):
        # Clear stored gradient
        model.zero_grad()
        # Don't do this if you want your LSTM to be stateful
        # Otherwise the hidden state should be cleaned up at each time step for prediction (we don't clear hidden state in our forward function)
        # see example from (https://github.com/pytorch/examples/blob/master/time_sequence_prediction/train.py)
        # model.hidden = model.init_hidden()
        # based on the design of this module, the hidden states and cell states are initialized when the module is called.
        for i in np.arange(1,sequence_len-test_year*12*4 + step + step_lead,1): # here i is actually the time step (index) of prediction, we use var[:i] to predict var[i]
            #############################################################################
            ###############           before time of prediction           ###############
            #############################################################################
            if i <= (sequence_len-test_year*12*4 + step):
                # create variables
                x_input = np.stack((sic_exp_norm[i-1,:,:],
                                    choice_exp_norm[i-1,:,:],
                                    month_exp[i-1,:,:])) #vstack,hstack,dstack
                x_var_pred = torch.autograd.Variable(torch.Tensor(x_input).view(-1,input_channels,height,width),
                                                     requires_grad=False).cuda()
                # make prediction
                last_pred, _ = model(x_var_pred, i-1, training=False)
                # record the real prediction after the time of prediction
                if i == (sequence_len-test_year*12*4 + step):
                    lead = 0
                    # GPU data should be transferred to CPU
                    lead_pred_sic[step,0,:,:] = last_pred[0,0,:,:].cpu().data.numpy()
            #############################################################################
            ###############            after time of prediction           ###############
            #############################################################################
            else:
                lead += 1
                # prepare predictor
                if i <= sequence_len:
                    # use the predicted data to make new prediction
                    x_input = np.stack((lead_pred_sic[step,i-(sequence_len-test_year*12*4 + step +1),:,:],
                                        choice_exp_norm[i-1,:,:],
                                        month_exp[i-1,:,:])) #vstack,hstack,dstack
                else: # choice_exp_norm out of range, use the last value
                    x_input = np.stack((lead_pred_sic[step,i-(sequence_len-test_year*12*4 + step +1),:,:],
                                        choice_exp_norm[-1,:,:],
                                        month_exp[i-1,:,:])) #vstack,hstack,dstack                    
                x_var_pred = torch.autograd.Variable(torch.Tensor(x_input).view(-1,input_channels,height,width),
                                                     requires_grad=False).cuda()        
                # make prediction
                last_pred, _ = model(x_var_pred, i-1, training=False)
                # record the prediction
                lead_pred_sic[step,lead,:,:] = last_pred[0,0,:,:].cpu().data.numpy()

In [ ]:
def week2month(series, m):
    """
    Select certain month from yearly data at weekly resolution.
    Parameters
    ----------
    series : array-like
        Three-dimensional numeric arrays with time as the first dimenison [time, lat, lon]
    m: int
        Month (from 1 to 12).
    """
    time_year, lat, lon = series.shape
    time_month = time_year // 12
    series_month = np.zeros((time_month, lat, lon), dtype=float)
    series_month[::4,:,:] = series[(m-1)*4::48,:,:]
    series_month[1::4,:,:] = series[(m-1)*4+1::48,:,:]
    series_month[2::4,:,:] = series[(m-1)*4+2::48,:,:]
    series_month[3::4,:,:] = series[(m-1)*4+3::48,:,:]
    
    return series_month

In [ ]:
    #################################################################################
    ########                performance evaluation with RMSE                 ########
    ########              RMSE over time, and sum over domain                ########
    #################################################################################
    sequence_len, height, width = sic_exp_norm.shape
    print('##############################################################')
    print('############   start prediction with climatology  ############')
    print('##############################################################')
    # compute climatology
    climatology = np.zeros((48, height, width),dtype=float)
    for i in range(48):
        climatology[i,:,:] = np.mean(sic_exp_norm[i::48,:,:],axis=0)
    # repeat this climatology and calculate the RMSE
    climatology_seq = np.tile(climatology,(test_year,1,1))
    RMSE_climatology, RMSE_climatology_std  = RMSE(climatology_seq * sic_max,sic_exp_norm[-test_year*12*4:,:,:] * sic_max)
    RMSE_climatology = np.mean(RMSE_climatology)
    RMSE_climatology_std = np.mean(RMSE_climatology_std)
    print('##############################################################')
    print('############   start prediction with persistence  ############')
    print('##############################################################')
    RMSE_persist_0, RMSE_persist_0_std = RMSE(sic_exp_norm[-test_year*12*4-1:-1,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4:,:,:] * sic_max)
    RMSE_persist_1, RMSE_persist_1_std = RMSE(sic_exp_norm[-test_year*12*4-1:-2,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+1:,:,:] * sic_max)
    RMSE_persist_2, RMSE_persist_2_std = RMSE(sic_exp_norm[-test_year*12*4-1:-3,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+2:,:,:] * sic_max)
    RMSE_persist_3, RMSE_persist_3_std = RMSE(sic_exp_norm[-test_year*12*4-1:-4,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+3:,:,:] * sic_max)
    RMSE_persist_4, RMSE_persist_4_std = RMSE(sic_exp_norm[-test_year*12*4-1:-5,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+4:,:,:] * sic_max)
    RMSE_persist_5, RMSE_persist_5_std = RMSE(sic_exp_norm[-test_year*12*4-1:-6,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+5:,:,:] * sic_max)
    RMSE_persist_6, RMSE_persist_6_std = RMSE(sic_exp_norm[-test_year*12*4-1:-7,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+6:,:,:] * sic_max)
    RMSE_persist_7, RMSE_persist_7_std = RMSE(sic_exp_norm[-test_year*12*4-1:-8,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+7:,:,:] * sic_max)
    RMSE_persist_8, RMSE_persist_8_std = RMSE(sic_exp_norm[-test_year*12*4-1:-9,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+8:,:,:] * sic_max)
    RMSE_persist_9, RMSE_persist_9_std = RMSE(sic_exp_norm[-test_year*12*4-1:-10,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+9:,:,:] * sic_max)
    RMSE_persist_10, RMSE_persist_10_std = RMSE(sic_exp_norm[-test_year*12*4-1:-11,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+10:,:,:] * sic_max)
    RMSE_persist_11, RMSE_persist_11_std = RMSE(sic_exp_norm[-test_year*12*4-1:-12,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+11:,:,:] * sic_max)
    RMSE_persist_12, RMSE_persist_12_std = RMSE(sic_exp_norm[-test_year*12*4-1:-13,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+12:,:,:] * sic_max)
    RMSE_persist_13, RMSE_persist_13_std = RMSE(sic_exp_norm[-test_year*12*4-1:-14,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+13:,:,:] * sic_max)
    RMSE_persist_14, RMSE_persist_14_std = RMSE(sic_exp_norm[-test_year*12*4-1:-15,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+14:,:,:] * sic_max)
    RMSE_persist_15, RMSE_persist_15_std = RMSE(sic_exp_norm[-test_year*12*4-1:-16,:,:] * sic_max,
                                              sic_exp_norm[-test_year*12*4+15:,:,:] * sic_max)
    print('##############################################################')
    print('########   land-sea correction for sic prediction    #########')
    print('##############################################################')
    # correction for float point at 0
    lead_pred_sic[lead_pred_sic<0] = 0
    # extend the dimension of sea ice mask
    sea_ice_mask_test = np.repeat(sea_ice_mask_barents[np.newaxis,:,:],test_year*48,0)
    # correct the land cells in the prediction
    for i in range(step_lead):
        lead_pred_sic[:,i,:,:] = lead_pred_sic[:,i,:,:] * sea_ice_mask_test
    print('#################################################################################')
    print('############   evaluation and statistical matrix for the entire year ############')
    print('#################################################################################')
    RMSE_ConvLSTM_0, RMSE_ConvLSTM_0_std = RMSE(lead_pred_sic[:,0,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4:,:,:] * sic_max)
    RMSE_ConvLSTM_1, RMSE_ConvLSTM_1_std = RMSE(lead_pred_sic[:-1,1,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+1:,:,:] * sic_max)
    RMSE_ConvLSTM_2, RMSE_ConvLSTM_2_std = RMSE(lead_pred_sic[:-2,2,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+2:,:,:] * sic_max)
    RMSE_ConvLSTM_3, RMSE_ConvLSTM_3_std = RMSE(lead_pred_sic[:-3,3,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+3:,:,:] * sic_max)
    RMSE_ConvLSTM_4, RMSE_ConvLSTM_4_std = RMSE(lead_pred_sic[:-4,4,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+4:,:,:] * sic_max)
    RMSE_ConvLSTM_5, RMSE_ConvLSTM_5_std = RMSE(lead_pred_sic[:-5,5,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+5:,:,:] * sic_max)
    RMSE_ConvLSTM_6, RMSE_ConvLSTM_6_std = RMSE(lead_pred_sic[:-6,6,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+6:,:,:] * sic_max)
    RMSE_ConvLSTM_7, RMSE_ConvLSTM_7_std = RMSE(lead_pred_sic[:-7,7,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+7:,:,:] * sic_max)
    RMSE_ConvLSTM_8, RMSE_ConvLSTM_8_std = RMSE(lead_pred_sic[:-8,8,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+8:,:,:] * sic_max)
    RMSE_ConvLSTM_9, RMSE_ConvLSTM_9_std = RMSE(lead_pred_sic[:-9,9,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+9:,:,:] * sic_max)
    RMSE_ConvLSTM_10, RMSE_ConvLSTM_10_std = RMSE(lead_pred_sic[:-10,10,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+10:,:,:] * sic_max)
    RMSE_ConvLSTM_11, RMSE_ConvLSTM_11_std = RMSE(lead_pred_sic[:-11,11,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+11:,:,:] * sic_max)
    RMSE_ConvLSTM_12, RMSE_ConvLSTM_12_std = RMSE(lead_pred_sic[:-12,12,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+12:,:,:] * sic_max)
    RMSE_ConvLSTM_13, RMSE_ConvLSTM_13_std = RMSE(lead_pred_sic[:-13,13,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+13:,:,:] * sic_max)
    RMSE_ConvLSTM_14, RMSE_ConvLSTM_14_std = RMSE(lead_pred_sic[:-14,14,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+14:,:,:] * sic_max)
    RMSE_ConvLSTM_15, RMSE_ConvLSTM_15_std = RMSE(lead_pred_sic[:-15,15,:,:] * sic_max,
                                                sic_exp_norm[-test_year*12*4+15:,:,:] * sic_max)
    print("*******************     Lead time 0     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_0),"+-",np.mean(RMSE_ConvLSTM_0_std))
    print("Mean RMSE with testing data - Climatology")
    print(RMSE_climatology,"+-",RMSE_climatology_std)
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_0),"+-",np.mean(RMSE_persist_0_std))
    print("*******************     Lead time 1     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_1),"+-",np.mean(RMSE_ConvLSTM_1_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_1),"+-",np.mean(RMSE_persist_1_std))
    print("*******************     Lead time 2     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_2),"+-",np.mean(RMSE_ConvLSTM_2_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_2),"+-",np.mean(RMSE_persist_2_std))
    print("*******************     Lead time 3     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_3),"+-",np.mean(RMSE_ConvLSTM_3_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_3),"+-",np.mean(RMSE_persist_3_std))
    print("*******************     Lead time 4     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_4),"+-",np.mean(RMSE_ConvLSTM_4_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_4),"+-",np.mean(RMSE_persist_4_std))
    print("*******************     Lead time 5     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_5),"+-",np.mean(RMSE_ConvLSTM_5_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_5),"+-",np.mean(RMSE_persist_5_std))
    print("*******************     Lead time 6     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_6),"+-",np.mean(RMSE_ConvLSTM_6_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_6),"+-",np.mean(RMSE_persist_6_std))
    print("*******************     Lead time 7     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_7),"+-",np.mean(RMSE_ConvLSTM_7_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_7),"+-",np.mean(RMSE_persist_7_std))
    print("*******************     Lead time 8     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_8),"+-",np.mean(RMSE_ConvLSTM_8_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_8),"+-",np.mean(RMSE_persist_8_std))
    print("*******************     Lead time 9     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_9),"+-",np.mean(RMSE_ConvLSTM_9_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_9),"+-",np.mean(RMSE_persist_9_std))
    print("*******************     Lead time 10     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_10),"+-",np.mean(RMSE_ConvLSTM_10_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_10),"+-",np.mean(RMSE_persist_10_std))
    print("*******************     Lead time 11     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_11),"+-",np.mean(RMSE_ConvLSTM_11_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_11),"+-",np.mean(RMSE_persist_11_std))
    print("*******************     Lead time 12     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_12),"+-",np.mean(RMSE_ConvLSTM_12_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_12),"+-",np.mean(RMSE_persist_12_std))
    print("*******************     Lead time 13     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_13),"+-",np.mean(RMSE_ConvLSTM_13_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_13),"+-",np.mean(RMSE_persist_13_std))
    print("*******************     Lead time 14     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_14),"+-",np.mean(RMSE_ConvLSTM_14_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_14),"+-",np.mean(RMSE_persist_14_std))
    print("*******************     Lead time 15     *******************")
    print("Mean RMSE with testing data - ConvLSTM")
    print(np.mean(RMSE_ConvLSTM_15),"+-",np.mean(RMSE_ConvLSTM_15_std))
    print("Mean RMSE with testing data - Persistence")
    print(np.mean(RMSE_persist_15),"+-",np.mean(RMSE_persist_15_std))
    print('##############################################################')
    print('############           create a txt file          ############')
    print('##############################################################')
    f = open(os.path.join(output_path,"report_rmse_pred_16weeks.txt"),"w+")
    f.write("############   evaluation of prediction   ############\n")
    f.write("############   Total RMSE with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_0),np.mean(RMSE_ConvLSTM_0_std)))
    f.write("RMSE - Climatology    {} + - {}\n".format(RMSE_climatology, RMSE_climatology_std))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_0),np.mean(RMSE_persist_0_std)))
    f.write("*******************     Lead time 1     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_1),np.mean(RMSE_ConvLSTM_1_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_1),np.mean(RMSE_persist_1_std)))
    f.write("*******************     Lead time 2     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_2),np.mean(RMSE_ConvLSTM_2_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_2),np.mean(RMSE_persist_2_std)))
    f.write("*******************     Lead time 3     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_3),np.mean(RMSE_ConvLSTM_3_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_3),np.mean(RMSE_persist_3_std)))
    f.write("*******************     Lead time 4     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_4),np.mean(RMSE_ConvLSTM_4_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_4),np.mean(RMSE_persist_4_std)))
    f.write("*******************     Lead time 5     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_5),np.mean(RMSE_ConvLSTM_5_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_5),np.mean(RMSE_persist_5_std)))
    f.write("*******************     Lead time 6     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_6),np.mean(RMSE_ConvLSTM_6_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_6),np.mean(RMSE_persist_6_std)))
    f.write("*******************     Lead time 7     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_7),np.mean(RMSE_ConvLSTM_7_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_7),np.mean(RMSE_persist_7_std)))
    f.write("*******************     Lead time 8     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_8),np.mean(RMSE_ConvLSTM_8_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_8),np.mean(RMSE_persist_8_std)))
    f.write("*******************     Lead time 9     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_9),np.mean(RMSE_ConvLSTM_9_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_9),np.mean(RMSE_persist_9_std)))
    f.write("*******************     Lead time 10     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_10),np.mean(RMSE_ConvLSTM_10_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_10),np.mean(RMSE_persist_10_std)))
    f.write("*******************     Lead time 11     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_11),np.mean(RMSE_ConvLSTM_11_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_11),np.mean(RMSE_persist_11_std)))
    f.write("*******************     Lead time 12     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_12),np.mean(RMSE_ConvLSTM_12_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_12),np.mean(RMSE_persist_12_std)))
    f.write("*******************     Lead time 13     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_13),np.mean(RMSE_ConvLSTM_13_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_13),np.mean(RMSE_persist_13_std)))
    f.write("*******************     Lead time 14     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_14),np.mean(RMSE_ConvLSTM_14_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_14),np.mean(RMSE_persist_14_std)))
    f.write("*******************     Lead time 15     *******************\n")
    f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(RMSE_ConvLSTM_15),np.mean(RMSE_ConvLSTM_15_std)))
    f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(RMSE_persist_15),np.mean(RMSE_persist_15_std)))
    f.write("-\n")
    f.write("-\n")
    f.write("-\n")
    f.write("############   Monthly RMSE with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    for i in np.arange(1,13,1):
        ConvLSTM_monthly_series = week2month(lead_pred_sic[:,0,:,:], i)
        persist_monthly_series = week2month(sic_exp_norm[-test_year*12*4-1:-1,:,:], i)
        climatology_monthly_series = week2month(climatology_seq, i)
        truth_monthly_series = week2month(sic_exp_norm[-test_year*12*4:,:,:], i)
        
        rmse_ConvLSTM_monthly, rmse_ConvLSTM_monthly_std = RMSE(ConvLSTM_monthly_series * sic_max,truth_monthly_series * sic_max)
        rmse_persist_monthly, rmse_persist_monthly_std = RMSE(persist_monthly_series * sic_max,truth_monthly_series * sic_max)
        rmse_climatology_monthly, rmse_climatology_monthly_std = RMSE(climatology_monthly_series * sic_max,truth_monthly_series * sic_max)
        
        print("*******************    {}     *******************".format(i))
        print("RMSE - ConvLSTM       {} + - {}".format(np.mean(rmse_ConvLSTM_monthly), np.mean(rmse_ConvLSTM_monthly_std)))
        print("RMSE - Climatology    {} + - {}".format(np.mean(rmse_climatology_monthly), np.mean(rmse_climatology_monthly_std)))
        print("RMSE - Persistence    {} + - {}".format(np.mean(rmse_persist_monthly), np.mean(rmse_persist_monthly_std)))
        f.write("*******************    {}     *******************\n".format(i))
        f.write("RMSE - ConvLSTM       {} + - {}\n".format(np.mean(rmse_ConvLSTM_monthly), np.mean(rmse_ConvLSTM_monthly_std)))
        f.write("RMSE - Climatology    {} + - {}\n".format(np.mean(rmse_climatology_monthly), np.mean(rmse_climatology_monthly_std)))
        f.write("RMSE - Persistence    {} + - {}\n".format(np.mean(rmse_persist_monthly), np.mean(rmse_persist_monthly_std)))
        
    #f.write("*******************     Lead time 6     *******************\n")
    f.close()

In [ ]:
    print ('*******************  module for calculating accuracy/recall/precision  *********************')
    def accuracy(pred, label):
        #print("Input size must be [seq, lat, lon]")
        seq, lat, lon = pred.shape
        boolean = (pred==label)
        accu_seq = np.mean(np.mean(boolean.astype(float),2),1)
        accu_spa = np.mean(boolean.astype(float),0)
        
        return accu_seq, accu_spa

In [ ]:
    #################################################################################
    ########          transfer the sea ice fields into binary data           ########
    #################################################################################
    # ice concentration below the threshold is regarded as no ice, the value is from
    # https://nsidc.org/cryosphere/seaice/data/terminology.html
    criterion_0 = 0.15 
    # remove the area weight
    sic_exp_denorm = np.zeros(sic_exp_norm.shape, dtype=float)
    lead_pred_sic_denorm = np.zeros(lead_pred_sic.shape, dtype=float)
    for i in np.arange(height):
        lead_pred_sic_denorm[:,:,i,:] = lead_pred_sic[:,:,i,:] /dx[i+12] * dx[35]
        sic_exp_denorm[:,i,:] = sic_exp_norm[:,i,:] / dx[i+12] * dx[35]
    # turn sea ice fields into binary data
    lead_pred_sic_bin = lead_pred_sic_denorm[:]
    sic_exp_bin = sic_exp_denorm[:]
    lead_pred_sic_bin[lead_pred_sic_bin <= criterion_0] = 0
    lead_pred_sic_bin[lead_pred_sic_bin > criterion_0] = 1
    sic_exp_bin[sic_exp_bin <= criterion_0] = 0
    sic_exp_bin[sic_exp_bin > criterion_0] = 1
    # turn matrix into int
    lead_pred_sic_bin = lead_pred_sic_bin.astype(int)
    sic_exp_bin = sic_exp_bin.astype(int)

In [ ]:
    #################################################################################
    ########              performance evaluation with accuracy               ########
    #################################################################################
    sequence_len, height, width = sic_exp_norm.shape
    print('##############################################################')
    print('############   start prediction with climatology  ############')
    print('##############################################################')
    # compute climatology
    climatology = np.zeros((48, height, width),dtype=float)
    for i in range(48):
        climatology[i,:,:] = np.mean(sic_exp_norm[i::48,:,:],axis=0)
    # repeat this climatology and calculate the RMSE
    climatology_seq = np.tile(climatology,(test_year,1,1))
    # turn into binary
    climatology_denorm = np.zeros(climatology_seq.shape, dtype=float)
    for i in np.arange(height):
        climatology_denorm[:,i,:] = climatology_seq[:,i,:] / dx[i+12] * dx[35]    
    climatology_denorm[climatology_denorm <= criterion_0] = 0
    climatology_denorm[climatology_denorm > criterion_0] = 1
    # turn matrix into int
    climatology_denorm = climatology_denorm.astype(int)
    
    accu_climatology, _ = accuracy(climatology_denorm, sic_exp_bin[-test_year*12*4:,:,:])
    print('##############################################################')
    print('############   start prediction with persistence  ############')
    print('##############################################################')
    accu_persist_0, _ = accuracy(sic_exp_bin[-test_year*12*4-1:-1,:,:],sic_exp_bin[-test_year*12*4:,:,:])
    accu_persist_1, _ = accuracy(sic_exp_bin[-test_year*12*4-1:-2,:,:],sic_exp_bin[-test_year*12*4+1:,:,:])
    accu_persist_2, _ = accuracy(sic_exp_bin[-test_year*12*4-1:-3,:,:],sic_exp_bin[-test_year*12*4+2:,:,:])
    accu_persist_3, _ = accuracy(sic_exp_bin[-test_year*12*4-1:-4,:,:],sic_exp_bin[-test_year*12*4+3:,:,:])
    accu_persist_4, _ = accuracy(sic_exp_bin[-test_year*12*4-1:-5,:,:],sic_exp_bin[-test_year*12*4+4:,:,:])
    accu_persist_5, _ = accuracy(sic_exp_bin[-test_year*12*4-1:-6,:,:],sic_exp_bin[-test_year*12*4+5:,:,:])
    print('##############################################################')
    print('########   start prediction with linear regression   #########')
    print('##############################################################')
    
    print('#################################################################################')
    print('############   evaluation and statistical matrix for the entire year ############')
    print('#################################################################################')
    accu_ConvLSTM_0, _ = accuracy(lead_pred_sic_bin[:,0,:,:],sic_exp_bin[-test_year*12*4:,:,:])
    accu_ConvLSTM_1, _ = accuracy(lead_pred_sic_bin[:-1,1,:,:],sic_exp_bin[-test_year*12*4+1:,:,:])
    accu_ConvLSTM_2, _ = accuracy(lead_pred_sic_bin[:-2,2,:,:],sic_exp_bin[-test_year*12*4+2:,:,:])
    accu_ConvLSTM_3, _ = accuracy(lead_pred_sic_bin[:-3,3,:,:],sic_exp_bin[-test_year*12*4+3:,:,:])
    accu_ConvLSTM_4, _ = accuracy(lead_pred_sic_bin[:-4,4,:,:],sic_exp_bin[-test_year*12*4+4:,:,:])
    accu_ConvLSTM_5, _ = accuracy(lead_pred_sic_bin[:-5,5,:,:],sic_exp_bin[-test_year*12*4+5:,:,:])
    print("*******************     Lead time 0     *******************")
    print("Total accuracy with testing data - ConvLSTM")
    print(np.mean(accu_ConvLSTM_0))
    print("Total accuracy with testing data - Climatology")
    print(np.mean(accu_climatology))
    print("Total accuracy with testing data - Persistence")
    print(np.mean(accu_persist_0))
    print("*******************     Lead time 1     *******************")
    print("Total accuracy with testing data - ConvLSTM")
    print(np.mean(accu_ConvLSTM_1))
    print("Total accuracy with testing data - Persistence")
    print(np.mean(accu_persist_1))
    print("*******************     Lead time 2     *******************")
    print("Total accuracy with testing data - ConvLSTM")
    print(np.mean(accu_ConvLSTM_2))
    print("Total accuracy with testing data - Persistence")
    print(np.mean(accu_persist_2))
    print("*******************     Lead time 3     *******************")
    print("Total accuracy with testing data - ConvLSTM")
    print(np.mean(accu_ConvLSTM_3))
    print("Total accuracy with testing data - Persistence")
    print(np.mean(accu_persist_3))
    print("*******************     Lead time 4     *******************")
    print("Total accuracy with testing data - ConvLSTM")
    print(np.mean(accu_ConvLSTM_4))
    print("Total accuracy with testing data - Persistence")
    print(np.mean(accu_persist_4))
    print("*******************     Lead time 5     *******************")
    print("Total accuracy with testing data - ConvLSTM")
    print(np.mean(accu_ConvLSTM_5))
    print("Total accuracy with testing data - Persistence")
    print(np.mean(accu_persist_5))
    print('##############################################################')
    print('############           create a txt file          ############')
    print('##############################################################')
    f = open(os.path.join(output_path,"report_accuracy_pred_bin.txt"),"w+")
    f.write("############   evaluation of prediction   ############\n")
    f.write("############   Total accuracy with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    f.write("accuracy - ConvLSTM       {}\n".format(np.mean(accu_ConvLSTM_0)))
    f.write("accuracy - Climatology    {}\n".format(np.mean(accu_climatology)))
    f.write("accuracy - Persistence    {}\n".format(np.mean(accu_persist_0)))
    f.write("*******************     Lead time 1     *******************\n")
    f.write("accuracy - ConvLSTM       {}\n".format(np.mean(accu_ConvLSTM_1)))
    f.write("accuracy - Persistence    {}\n".format(np.mean(accu_persist_1)))
    f.write("*******************     Lead time 2     *******************\n")
    f.write("accuracy - ConvLSTM       {}\n".format(np.mean(accu_ConvLSTM_2)))
    f.write("accuracy - Persistence    {}\n".format(np.mean(accu_persist_2)))
    f.write("*******************     Lead time 3     *******************\n")
    f.write("accuracy - ConvLSTM       {}\n".format(np.mean(accu_ConvLSTM_3)))
    f.write("accuracy - Persistence    {}\n".format(np.mean(accu_persist_3)))
    f.write("*******************     Lead time 4     *******************\n")
    f.write("accuracy - ConvLSTM       {}\n".format(np.mean(accu_ConvLSTM_4)))
    f.write("accuracy - Persistence    {}\n".format(np.mean(accu_persist_4)))
    f.write("*******************     Lead time 5     *******************\n")
    f.write("accuracy - ConvLSTM       {}\n".format(np.mean(accu_ConvLSTM_5)))
    f.write("accuracy - Persistence    {}\n".format(np.mean(accu_persist_5)))
    f.write("-\n")
    f.write("-\n")
    f.write("-\n")
    f.write("############   Monthly accuracy with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    for i in np.arange(1,13,1):
        ConvLSTM_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        persist_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        climatology_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        truth_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        for j in np.arange(4):
            ConvLSTM_monthly_series[j::4,:,:] = lead_pred_sic_bin[(i-1)*4+j::48,0,:,:]
            persist_monthly_series[j::4,:,:] = sic_exp_bin[-test_year*12*4-1+(i-1)*4+j:-1:48,:,:]
            climatology_monthly_series[j::4,:,:] = climatology_denorm[(i-1)*4+j::48,:,:]
            truth_monthly_series[j::4,:,:] = sic_exp_bin[-test_year*12*4+(i-1)*4+j::48,:,:]
        print("*******************    {}     *******************".format(i))
        print("accuracy - ConvLSTM       {}".format(np.mean(accuracy(ConvLSTM_monthly_series,truth_monthly_series)[0])))
        print("accuracy - Climatology    {}".format(np.mean(accuracy(climatology_monthly_series,truth_monthly_series)[0])))
        print("accuracy - Persistence    {}".format(np.mean(accuracy(persist_monthly_series,truth_monthly_series)[0])))
        f.write("*******************    {}     *******************\n".format(i))
        f.write("accuracy - ConvLSTM       {}\n".format(np.mean(accuracy(ConvLSTM_monthly_series,truth_monthly_series)[0])))
        f.write("accuracy - Climatology    {}\n".format(np.mean(accuracy(climatology_monthly_series,truth_monthly_series)[0])))
        f.write("accuracy - Persistence    {}\n".format(np.mean(accuracy(persist_monthly_series,truth_monthly_series)[0])))
    f.write("*******************     Lead time 6     *******************\n")
    f.close()

In [ ]:
    print ('*******************  module for calculating recall/precision  *********************')
    # positive is sea ice = 1
    
    def recall(pred, label):
        """
        True positive / Total actual positive
        Input fields must contain only 0 / 1. 1 is positive.
        """
        #print("Input size must be [seq, lat, lon]")
        seq, lat, lon = pred.shape
        # initialize dummy matrix
        pred_dummy_1 = np.zeros(pred.shape,dtype=int)
        label_dummy_1 = np.zeros(label.shape,dtype=int)
        # True positive
        # create dummy matrix to save the labels
        pred_dummy_1[:] = pred[:]
        label_dummy_1[:] = label[:]
        # change the label of negative events
        pred_dummy_1[pred == 0] = 2
        label_dummy_1[label == 0] = 3
        # count True Positive events
        truePositive = (pred_dummy_1 == label_dummy_1)

        # initialize dummy matrix
        pred_dummy_2 = np.zeros(pred.shape,dtype=int)
        label_dummy_2 = np.zeros(label.shape,dtype=int)
        # False negative (is 1 but predict 0)
        # create dummy matrix to save the labels (reset dummy)
        pred_dummy_2[:] = pred[:]
        label_dummy_2[:] = label[:]
        pred_dummy_2[pred == 0] = 2
        label_dummy_2[label == 1] = 2
        # count False Positive events
        falseNegative = (pred_dummy_2 == label_dummy_2)

#         recall_seq = np.mean(np.mean(np.nan_to_num(truePositive.astype(float) / 
#                                     (truePositive.astype(float) + falseNegative.astype(float))),2),1)
        
        recall_seq = np.sum(np.sum(truePositive.astype(float),2),1) / (np.sum(np.sum(truePositive.astype(float),2),1) +
                                                                       np.sum(np.sum(falseNegative.astype(float),2),1))
        
#         recall_spa = np.mean(np.nan_to_num(truePositive.astype(float) / 
#                                           (truePositive.astype(float) + falseNegative.astype(float))),0)
        
        recall_spa = np.sum(truePositive.astype(float),0) / (np.sum(truePositive.astype(float),0) +
                                                             np.sum(falseNegative.astype(float),0))
        
        #return recall_seq, recall_spa
        return np.nan_to_num(recall_seq), np.nan_to_num(recall_spa)
    
    def precision(pred, label):
        """
        True positive / Total predicted positive
        Input fields must contain only 0 / 1. 1 is positive.
        """
        #print("Input size must be [seq, lat, lon]")
        seq, lat, lon = pred.shape
        # initialize dummy matrix
        pred_dummy_1 = np.zeros(pred.shape,dtype=int)
        label_dummy_1 = np.zeros(label.shape,dtype=int)
        # True positive
        # create dummy matrix to save the labels
        pred_dummy_1[:] = pred[:]
        label_dummy_1[:] = label[:]
        # change the label of negative events
        pred_dummy_1[pred == 0] = 2
        label_dummy_1[label == 0] = 3
        # count True Positive events
        truePositive = (pred_dummy_1 == label_dummy_1)

        # initialize dummy matrix
        pred_dummy_2 = np.zeros(pred.shape,dtype=int)
        label_dummy_2 = np.zeros(label.shape,dtype=int)
        # False positive (is 0 but predict 1)
        # create dummy matrix to save the labels (reset dummy)
        pred_dummy_2[:] = pred[:]
        label_dummy_2[:] = label[:]
        pred_dummy_2[pred == 1] = 2
        label_dummy_2[label == 0] = 2
        # count False Positive events
        falsePositive = (pred_dummy_2 == label_dummy_2)
        
        prec_seq = np.sum(np.sum(truePositive.astype(float),2),1) / (np.sum(np.sum(truePositive.astype(float),2),1) +
                                                                     np.sum(np.sum(falsePositive.astype(float),2),1))
        
        prec_spa = np.sum(truePositive.astype(float),0) / (np.sum(truePositive.astype(float),0) +
                                                           np.sum(falsePositive.astype(float),0))
        
        return np.nan_to_num(prec_seq), np.nan_to_num(prec_spa)

In [ ]:
    #################################################################################
    ########                performance evaluation with recall               ########
    #################################################################################
    sequence_len, height, width = sic_exp_norm.shape
    print('##############################################################')
    print('############   start prediction with climatology  ############')
    print('##############################################################')
    # compute climatology
    climatology = np.zeros((48, height, width),dtype=float)
    for i in range(48):
        climatology[i,:,:] = np.mean(sic_exp_norm[i::48,:,:],axis=0)
    # repeat this climatology and calculate the RMSE
    climatology_seq = np.tile(climatology,(test_year,1,1))
    # turn into binary
    climatology_denorm = np.zeros(climatology_seq.shape, dtype=float)
    for i in np.arange(height):
        climatology_denorm[:,i,:] = climatology_seq[:,i,:] / dx[i+12] * dx[35]    
    climatology_denorm[climatology_denorm <= criterion_0] = 0
    climatology_denorm[climatology_denorm > criterion_0] = 1
    # turn matrix into int
    climatology_denorm = climatology_denorm.astype(int)
    
    recall_climatology, _ = recall(climatology_denorm, sic_exp_bin[-test_year*12*4:,:,:])
    print('##############################################################')
    print('############   start prediction with persistence  ############')
    print('##############################################################')
    recall_persist_0, _ = recall(sic_exp_bin[-test_year*12*4-1:-1,:,:],sic_exp_bin[-test_year*12*4:,:,:])
    recall_persist_1, _ = recall(sic_exp_bin[-test_year*12*4-1:-2,:,:],sic_exp_bin[-test_year*12*4+1:,:,:])
    recall_persist_2, _ = recall(sic_exp_bin[-test_year*12*4-1:-3,:,:],sic_exp_bin[-test_year*12*4+2:,:,:])
    recall_persist_3, _ = recall(sic_exp_bin[-test_year*12*4-1:-4,:,:],sic_exp_bin[-test_year*12*4+3:,:,:])
    recall_persist_4, _ = recall(sic_exp_bin[-test_year*12*4-1:-5,:,:],sic_exp_bin[-test_year*12*4+4:,:,:])
    recall_persist_5, _ = recall(sic_exp_bin[-test_year*12*4-1:-6,:,:],sic_exp_bin[-test_year*12*4+5:,:,:])
    print('##############################################################')
    print('########   start prediction with linear regression   #########')
    print('##############################################################')
    
    print('#################################################################################')
    print('############   evaluation and statistical matrix for the entire year ############')
    print('#################################################################################')
    recall_ConvLSTM_0, _ = recall(lead_pred_sic_bin[:,0,:,:],sic_exp_bin[-test_year*12*4:,:,:])
    recall_ConvLSTM_1, _ = recall(lead_pred_sic_bin[:-1,1,:,:],sic_exp_bin[-test_year*12*4+1:,:,:])
    recall_ConvLSTM_2, _ = recall(lead_pred_sic_bin[:-2,2,:,:],sic_exp_bin[-test_year*12*4+2:,:,:])
    recall_ConvLSTM_3, _ = recall(lead_pred_sic_bin[:-3,3,:,:],sic_exp_bin[-test_year*12*4+3:,:,:])
    recall_ConvLSTM_4, _ = recall(lead_pred_sic_bin[:-4,4,:,:],sic_exp_bin[-test_year*12*4+4:,:,:])
    recall_ConvLSTM_5, _ = recall(lead_pred_sic_bin[:-5,5,:,:],sic_exp_bin[-test_year*12*4+5:,:,:])
    print("*******************     Lead time 0     *******************")
    print("Total recall with testing data - ConvLSTM")
    print(np.mean(recall_ConvLSTM_0))
    print("Total recall with testing data - Climatology")
    print(np.mean(recall_climatology))
    print("Total recall with testing data - Persistence")
    print(np.mean(recall_persist_0))
    print("*******************     Lead time 1     *******************")
    print("Total recall with testing data - ConvLSTM")
    print(np.mean(recall_ConvLSTM_1))
    print("Total recall with testing data - Persistence")
    print(np.mean(recall_persist_1))
    print("*******************     Lead time 2     *******************")
    print("Total recall with testing data - ConvLSTM")
    print(np.mean(recall_ConvLSTM_2))
    print("Total recall with testing data - Persistence")
    print(np.mean(recall_persist_2))
    print("*******************     Lead time 3     *******************")
    print("Total recall with testing data - ConvLSTM")
    print(np.mean(recall_ConvLSTM_3))
    print("Total recall with testing data - Persistence")
    print(np.mean(recall_persist_3))
    print("*******************     Lead time 4     *******************")
    print("Total recall with testing data - ConvLSTM")
    print(np.mean(recall_ConvLSTM_4))
    print("Total recall with testing data - Persistence")
    print(np.mean(recall_persist_4))
    print("*******************     Lead time 5     *******************")
    print("Total recall with testing data - ConvLSTM")
    print(np.mean(recall_ConvLSTM_5))
    print("Total recall with testing data - Persistence")
    print(np.mean(recall_persist_5))
    print('##############################################################')
    print('############           create a txt file          ############')
    print('##############################################################')
    f = open(os.path.join(output_path,"report_recall_pred_bin.txt"),"w+")
    f.write("############   evaluation of prediction   ############\n")
    f.write("############   Total recall with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    f.write("recall - ConvLSTM       {}\n".format(np.mean(recall_ConvLSTM_0)))
    f.write("recall - Climatology    {}\n".format(np.mean(recall_climatology)))
    f.write("recall - Persistence    {}\n".format(np.mean(recall_persist_0)))
    f.write("*******************     Lead time 1     *******************\n")
    f.write("recall - ConvLSTM       {}\n".format(np.mean(recall_ConvLSTM_1)))
    f.write("recall - Persistence    {}\n".format(np.mean(recall_persist_1)))
    f.write("*******************     Lead time 2     *******************\n")
    f.write("recall - ConvLSTM       {}\n".format(np.mean(recall_ConvLSTM_2)))
    f.write("recall - Persistence    {}\n".format(np.mean(recall_persist_2)))
    f.write("*******************     Lead time 3     *******************\n")
    f.write("recall - ConvLSTM       {}\n".format(np.mean(recall_ConvLSTM_3)))
    f.write("recall - Persistence    {}\n".format(np.mean(recall_persist_3)))
    f.write("*******************     Lead time 4     *******************\n")
    f.write("recall - ConvLSTM       {}\n".format(np.mean(recall_ConvLSTM_4)))
    f.write("recall - Persistence    {}\n".format(np.mean(recall_persist_4)))
    f.write("*******************     Lead time 5     *******************\n")
    f.write("recall - ConvLSTM       {}\n".format(np.mean(recall_ConvLSTM_5)))
    f.write("recall - Persistence    {}\n".format(np.mean(recall_persist_5)))
    f.write("-\n")
    f.write("-\n")
    f.write("-\n")
    f.write("############   Monthly accuracy with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    for i in np.arange(1,13,1):
        ConvLSTM_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        persist_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        climatology_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        truth_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        for j in np.arange(4):
            ConvLSTM_monthly_series[j::4,:,:] = lead_pred_sic_bin[(i-1)*4+j::48,0,:,:]
            persist_monthly_series[j::4,:,:] = sic_exp_bin[-test_year*12*4-1+(i-1)*4+j:-1:48,:,:]
            climatology_monthly_series[j::4,:,:] = climatology_denorm[(i-1)*4+j::48,:,:]
            truth_monthly_series[j::4,:,:] = sic_exp_bin[-test_year*12*4+(i-1)*4+j::48,:,:]
        print("*******************    {}     *******************".format(i))
        print("recall - ConvLSTM       {}".format(np.mean(recall(ConvLSTM_monthly_series,truth_monthly_series)[0])))
        print("recall - Climatology    {}".format(np.mean(recall(climatology_monthly_series,truth_monthly_series)[0])))
        print("recall - Persistence    {}".format(np.mean(recall(persist_monthly_series,truth_monthly_series)[0])))
        f.write("*******************    {}     *******************\n".format(i))
        f.write("recall - ConvLSTM       {}\n".format(np.mean(recall(ConvLSTM_monthly_series,truth_monthly_series)[0])))
        f.write("recall - Climatology    {}\n".format(np.mean(recall(climatology_monthly_series,truth_monthly_series)[0])))
        f.write("recall - Persistence    {}\n".format(np.mean(recall(persist_monthly_series,truth_monthly_series)[0])))
    f.write("*******************     Lead time 6     *******************\n")
    f.close()

In [ ]:
    #################################################################################
    ########              performance evaluation with precision              ########
    #################################################################################
    sequence_len, height, width = sic_exp_norm.shape
    print('##############################################################')
    print('############   start prediction with climatology  ############')
    print('##############################################################')
    # compute climatology
    climatology = np.zeros((48, height, width),dtype=float)
    for i in range(48):
        climatology[i,:,:] = np.mean(sic_exp_norm[i::48,:,:],axis=0)
    # repeat this climatology and calculate the RMSE
    climatology_seq = np.tile(climatology,(test_year,1,1))
    # turn into binary
    climatology_denorm = np.zeros(climatology_seq.shape, dtype=float)
    for i in np.arange(height):
        climatology_denorm[:,i,:] = climatology_seq[:,i,:] / dx[i+12] * dx[35]    
    climatology_denorm[climatology_denorm <= criterion_0] = 0
    climatology_denorm[climatology_denorm > criterion_0] = 1
    # turn matrix into int
    climatology_denorm = climatology_denorm.astype(int)
    
    prec_climatology, _ = precision(climatology_denorm, sic_exp_bin[-test_year*12*4:,:,:])
    print('##############################################################')
    print('############   start prediction with persistence  ############')
    print('##############################################################')
    prec_persist_0, _ = precision(sic_exp_bin[-test_year*12*4-1:-1,:,:],sic_exp_bin[-test_year*12*4:,:,:])
    prec_persist_1, _ = precision(sic_exp_bin[-test_year*12*4-1:-2,:,:],sic_exp_bin[-test_year*12*4+1:,:,:])
    prec_persist_2, _ = precision(sic_exp_bin[-test_year*12*4-1:-3,:,:],sic_exp_bin[-test_year*12*4+2:,:,:])
    prec_persist_3, _ = precision(sic_exp_bin[-test_year*12*4-1:-4,:,:],sic_exp_bin[-test_year*12*4+3:,:,:])
    prec_persist_4, _ = precision(sic_exp_bin[-test_year*12*4-1:-5,:,:],sic_exp_bin[-test_year*12*4+4:,:,:])
    prec_persist_5, _ = precision(sic_exp_bin[-test_year*12*4-1:-6,:,:],sic_exp_bin[-test_year*12*4+5:,:,:])
    print('##############################################################')
    print('########   start prediction with linear regression   #########')
    print('##############################################################')
    
    print('#################################################################################')
    print('############   evaluation and statistical matrix for the entire year ############')
    print('#################################################################################')
    prec_ConvLSTM_0, _ = precision(lead_pred_sic_bin[:,0,:,:],sic_exp_bin[-test_year*12*4:,:,:])
    prec_ConvLSTM_1, _ = precision(lead_pred_sic_bin[:-1,1,:,:],sic_exp_bin[-test_year*12*4+1:,:,:])
    prec_ConvLSTM_2, _ = precision(lead_pred_sic_bin[:-2,2,:,:],sic_exp_bin[-test_year*12*4+2:,:,:])
    prec_ConvLSTM_3, _ = precision(lead_pred_sic_bin[:-3,3,:,:],sic_exp_bin[-test_year*12*4+3:,:,:])
    prec_ConvLSTM_4, _ = precision(lead_pred_sic_bin[:-4,4,:,:],sic_exp_bin[-test_year*12*4+4:,:,:])
    prec_ConvLSTM_5, _ = precision(lead_pred_sic_bin[:-5,5,:,:],sic_exp_bin[-test_year*12*4+5:,:,:])
    print("*******************     Lead time 0     *******************")
    print("Total precision with testing data - ConvLSTM")
    print(np.mean(prec_ConvLSTM_0))
    print("Total precision with testing data - Climatology")
    print(np.mean(prec_climatology))
    print("Total precision with testing data - Persistence")
    print(np.mean(prec_persist_0))
    print("*******************     Lead time 1     *******************")
    print("Total precision with testing data - ConvLSTM")
    print(np.mean(prec_ConvLSTM_1))
    print("Total precision with testing data - Persistence")
    print(np.mean(prec_persist_1))
    print("*******************     Lead time 2     *******************")
    print("Total precision with testing data - ConvLSTM")
    print(np.mean(prec_ConvLSTM_2))
    print("Total precision with testing data - Persistence")
    print(np.mean(prec_persist_2))
    print("*******************     Lead time 3     *******************")
    print("Total precision with testing data - ConvLSTM")
    print(np.mean(prec_ConvLSTM_3))
    print("Total precision with testing data - Persistence")
    print(np.mean(prec_persist_3))
    print("*******************     Lead time 4     *******************")
    print("Total precision with testing data - ConvLSTM")
    print(np.mean(prec_ConvLSTM_4))
    print("Total precision with testing data - Persistence")
    print(np.mean(prec_persist_4))
    print("*******************     Lead time 5     *******************")
    print("Total precision with testing data - ConvLSTM")
    print(np.mean(prec_ConvLSTM_5))
    print("Total precision with testing data - Persistence")
    print(np.mean(prec_persist_5))
    print('##############################################################')
    print('############           create a txt file          ############')
    print('##############################################################')
    f = open(os.path.join(output_path,"report_precision_pred_bin.txt"),"w+")
    f.write("############   evaluation of prediction   ############\n")
    f.write("############   Total precision with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    f.write("precision - ConvLSTM       {}\n".format(np.mean(prec_ConvLSTM_0)))
    f.write("precision - Climatology    {}\n".format(np.mean(prec_climatology)))
    f.write("precision - Persistence    {}\n".format(np.mean(prec_persist_0)))
    f.write("*******************     Lead time 1     *******************\n")
    f.write("precision - ConvLSTM       {}\n".format(np.mean(prec_ConvLSTM_1)))
    f.write("precision - Persistence    {}\n".format(np.mean(prec_persist_1)))
    f.write("*******************     Lead time 2     *******************\n")
    f.write("precision - ConvLSTM       {}\n".format(np.mean(prec_ConvLSTM_2)))
    f.write("precision - Persistence    {}\n".format(np.mean(prec_persist_2)))
    f.write("*******************     Lead time 3     *******************\n")
    f.write("precision - ConvLSTM       {}\n".format(np.mean(prec_ConvLSTM_3)))
    f.write("precision - Persistence    {}\n".format(np.mean(prec_persist_3)))
    f.write("*******************     Lead time 4     *******************\n")
    f.write("precision - ConvLSTM       {}\n".format(np.mean(prec_ConvLSTM_4)))
    f.write("precision - Persistence    {}\n".format(np.mean(prec_persist_4)))
    f.write("*******************     Lead time 5     *******************\n")
    f.write("precision - ConvLSTM       {}\n".format(np.mean(prec_ConvLSTM_5)))
    f.write("precision - Persistence    {}\n".format(np.mean(prec_persist_5)))
    f.write("-\n")
    f.write("-\n")
    f.write("-\n")
    f.write("############   Monthly accuracy with testing data   ############\n")
    f.write("*******************     Lead time 0     *******************\n")
    for i in np.arange(1,13,1):
        ConvLSTM_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        persist_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        climatology_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        truth_monthly_series = np.zeros((test_year*4,height, width),dtype=float)
        for j in np.arange(4):
            ConvLSTM_monthly_series[j::4,:,:] = lead_pred_sic_bin[(i-1)*4+j::48,0,:,:]
            persist_monthly_series[j::4,:,:] = sic_exp_bin[-test_year*12*4-1+(i-1)*4+j:-1:48,:,:]
            climatology_monthly_series[j::4,:,:] = climatology_denorm[(i-1)*4+j::48,:,:]
            truth_monthly_series[j::4,:,:] = sic_exp_bin[-test_year*12*4+(i-1)*4+j::48,:,:]
        print("*******************    {}     *******************".format(i))
        print("precision - ConvLSTM       {}".format(np.mean(precision(ConvLSTM_monthly_series,truth_monthly_series)[0])))
        print("precision - Climatology    {}".format(np.mean(precision(climatology_monthly_series,truth_monthly_series)[0])))
        print("precision - Persistence    {}".format(np.mean(precision(persist_monthly_series,truth_monthly_series)[0])))
        f.write("*******************    {}     *******************\n".format(i))
        f.write("precision - ConvLSTM       {}\n".format(np.mean(precision(ConvLSTM_monthly_series,truth_monthly_series)[0])))
        f.write("precision - Climatology    {}\n".format(np.mean(precision(climatology_monthly_series,truth_monthly_series)[0])))
        f.write("precision - Persistence    {}\n".format(np.mean(precision(persist_monthly_series,truth_monthly_series)[0])))
    f.write("*******************     Lead time 6     *******************\n")
    f.close()